In [3]:
import pandas as pd
import os                        ###Se importan las librerías necesarias para el tratamiento de datos y gekko para realizar la optimización
import numpy as np
from gekko import GEKKO    
m = GEKKO(remote=False)

datos_generales = pd.read_excel(r"C:\Optimizacion\Data_Optimizacion.xlsx",sheet_name="Datos de entrada generales")
datos_zonas = pd.read_excel(r"C:\Optimizacion\Data_Optimizacion.xlsx",sheet_name="Datos de entrada")  ###Se importa la data de entrada del modelo del archivo
datos_tiendas = pd.read_excel(r"C:\Optimizacion\Data_Optimizacion.xlsx",sheet_name="Datos_tienda")    ###Data_Optimizacion 

cantidad_tiendas = datos_generales.iloc[0,0]
cantidad_zonas = np.round(datos_generales.iloc[0,5],decimals =0)
traslados = datos_generales.iloc[0,6]                               ###Se integra al modelo la data inicial de cantidad de tipos de tiendas
traslados = np.round(traslados  + 1,decimals = 1)                   ###cantidad de zonas y porcentaje de traslados
cantidad_zonas  = int(cantidad_zonas)
cantidad_tiendas = int(cantidad_tiendas)

Mix_obj_zona = []
Demanda_zona = []
for i in range(0,cantidad_zonas):
    Mix_obj_zona.append(datos_zonas.iloc[i,1])     ###Se integra al modelo los limites de las restricciones de mix y de capacidad
    Demanda_zona.append(datos_zonas.iloc[i,2])  
    
limite_restriccion_mix = []

for elemento in Mix_obj_zona:
    elemento = np.round(elemento*traslados,decimals = 0)
    elemento = int(elemento)
    limite_restriccion_mix.append(elemento)
    

cap_atencion_tienda  = []
mix_tienda = []                                              ###Se integra al modelo las capacidades de atención de las tiendas
                                                             ### y las capacidades de mix
for i in range(0,cantidad_tiendas):
    cap_atencion_tienda.append(datos_tiendas.iloc[i,1])
    mix_tienda.append(datos_tiendas.iloc[i,2])
    
    
def funcion_objetivo(tiendas,desvCap_encima,desvMix_encima,mix_tienda,cantidad_tiendas):
    parte1 = 0
    parte2 = 0
    parte3 = 0                       ###Se definen las funciones objetivo y las restricciones del modelo de optimización
    
    for k in range(0,cantidad_tiendas):
        parte1 = tiendas[k]*desvCap_encima[k] + parte1 
        parte2 = tiendas[k]*desvMix_encima[k] + parte2
        parte3 = tiendas[k]*mix_tienda[k] + parte3
    
    return (parte1 + parte2 + parte3)
        
        
def restriccion_capacidad(tiendas,desvCap_encima,desvCap_bajo,cap_atencion_tienda,cantidad_tiendas):
    parte1R1 = 0
    parte2R1 = 0
    parte3R1 = 0
    
    for k in range(0,cantidad_tiendas):
        parte1R1 = tiendas[k]*cap_atencion_tienda[k] + parte1R1
        parte2R1= tiendas[k]*desvCap_bajo[k] + parte2R1
        parte3R1 = tiendas[k]*desvCap_encima[k] + parte3R1
        
    return (parte1R1 + parte2R1 - parte3R1)


def restriccion_mix(tiendas,desvMix_bajo,desvMix_encima,mix_tienda,cantidad_tiendas):
    parte1R2 = 0
    parte2R2 = 0
    parte3R2 = 0
    
    for k in range(0,cantidad_tiendas):
        parte1R2 = tiendas[k]*mix_tienda[k] + parte1R2
        parte2R2 = tiendas[k]*desvMix_bajo[k] + parte2R2
        parte3R2 = tiendas[k]*desvMix_encima[k] + parte3R2
        
    return (parte1R2 + parte2R2 - parte3R2)

def restriccion_capacidad_bajo(desvCap_bajo):
    parte1R3 = 0
    
    for k in range(0,cantidad_tiendas):
        parte1R3 = desvCap_bajo[k] + parte1R3
        
    return parte1R3



def restriccion_mix_bajo(desvMix_bajo):
    parte1R4 = 0
    
    for k in range(0,cantidad_tiendas):
        parte1R4 = desvMix_bajo[k] + parte1R4
        
    return parte1R4


nombres_zonas = []
T=np.empty(cantidad_zonas)      
z=[None]*len(T)

for j in range(0,cantidad_zonas):
    
    tiendas = m.Array(m.Var,cantidad_tiendas,lb=0,integer=True)
    desvCap_bajo = m.Array(m.Var,cantidad_tiendas,lb =0)
    desvCap_encima = m.Array(m.Var,cantidad_tiendas,lb =0)
    desvMix_bajo = m.Array(m.Var,cantidad_tiendas,lb  =0)
    desvMix_encima = m.Array(m.Var,cantidad_tiendas,lb =0)
    
    inicial_tiendas =[1,1,1,1,1]
    inicial_desvCap_bajo = [0,0,0,0,0]
    inicial_desvCap_encima = [0,0,0,0,0]
    inicial_desvMix_bajo = [0,0,0,0,0]
    inicial_desvMix_encima = [0,0,0,0,0]
    
    i = 0
    for tiendasi in tiendas:
        tiendasi.value = inicial_tiendas[i]
        tiendasi.lower = 0
        tiendasi.upper= 100
        i = i + 1
    
    i = 0
    for desvCap_bajoi in desvCap_bajo:
        desvCap_bajoi.value = inicial_desvCap_bajo[i]
        desvCap_bajoi.lower = 0
        desvCap_bajoi.upper = 1000000

    i=0
    for desvCap_encimai in desvCap_encima:
        desvCap_encimai.value = inicial_desvCap_encima[i]
        desvCap_encimai.lower = 0
        desvCap_encimai.upper = 1000000
    
    i=0
    for desvMix_bajoi in desvMix_bajo:
        desvMix_bajoi.value = inicial_desvMix_bajo[i]
        desvMix_bajoi.lower = 0
        desvMix_bajoi.upper = 1000000

    i=0
    for desvMix_encimai in desvMix_encima:
        desvMix_encimai.value = inicial_desvMix_encima[i]
        desvMix_encimai.lower = 0
        desvMix_encimai.upper = 1000000
        
    m.Equation(restriccion_capacidad(tiendas,desvCap_encima,desvCap_bajo,cap_atencion_tienda,cantidad_tiendas) == Demanda_zona[j])
    m.Equation(restriccion_mix(tiendas,desvMix_bajo,desvMix_encima,mix_tienda,cantidad_tiendas) == limite_restriccion_mix[j])
    m.Equation(restriccion_capacidad_bajo(desvCap_bajo) <= 0)
    m.Equation(restriccion_mix_bajo(desvMix_bajo)<=0)
    m.Minimize(funcion_objetivo(tiendas,desvCap_encima,desvMix_encima,mix_tienda,cantidad_tiendas))
    m.options.SOLVER = 1

    m.solver_options = ['minlp_maximum_iterations 5000', \
                    # minlp iterations with integer solution
                    'minlp_max_iter_with_int_sol 5000', \
                    # treat minlp as nlp
                    'minlp_as_nlp 100', \
                    # nlp sub-problem max iterations
                    'nlp_maximum_iterations 1000']

    m.solve()
    
    print("Respuesta Configuración Zona: " + str(j))
    print(tiendas)
    
    nuevos_val = []
    for a in range(0,len(tiendas)):
        nuevos_val.append(tiendas[a].value[0])
        
        
    z[j]=nuevos_val


nombres_zonas = []
for j in range(0,cantidad_zonas):
    nombre_zona = "Zona " + str(j)
    nombres_zonas.append(nombre_zona)
    
z = np.array(z)
z = np.transpose(z)
resultado= pd.DataFrame(z,columns = nombres_zonas )
print(resultado)
resultado.to_excel(r"C:\Optimizacion\Resultado_Optimizacion.xlsx")

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  27
   Intermediates:  0
   Connections  :  0
   Equations    :  5
   Residuals    :  5
 
 Number of state variables:    27
 Number of total equations: -  4
 Number of slack variables: -  2
 ---------------------------------------
 Degrees of freedom       :    21
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    3 Dpth:    0 Lvs:    3 Obj:  4.95E+03 Gap:       NaN
Iter:     2 I:  0 Tm:     -0.00 NLPi:    3 Dpth:    1 Lvs:    4 Obj:  8.38E+03 Gap:       NaN
Iter:     3 I:  0 Tm:     -0.00 NLPi:    3 Dpth:    1 Lvs:    5 Obj:  5.20E+03 Gap:     

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  81
   Intermediates:  0
   Connections  :  0
   Equations    :  15
   Residuals    :  15
 
 Number of state variables:    81
 Number of total equations: -  12
 Number of slack variables: -  6
 ---------------------------------------
 Degrees of freedom       :    63
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj:  2.04E+04 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi:    3 Dpth:    1 Lvs:    2 Obj:  2.04E+04 Gap:       NaN
Iter:     3 I:  0 Tm:      0.00 NLPi:    3 Dpth:    1 Lvs:    4 Obj:  5.22E+04 Gap:  

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  108
   Intermediates:  0
   Connections  :  0
   Equations    :  20
   Residuals    :  20
 
 Number of state variables:    108
 Number of total equations: -  16
 Number of slack variables: -  8
 ---------------------------------------
 Degrees of freedom       :    84
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj:  4.04E+04 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi:    2 Dpth:    1 Lvs:    2 Obj:  4.04E+04 Gap:       NaN
Iter:     3 I:  0 Tm:      0.00 NLPi:    6 Dpth:    1 Lvs:    4 Obj:  4.04E+04 Gap:

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  135
   Intermediates:  0
   Connections  :  0
   Equations    :  25
   Residuals    :  25
 
 Number of state variables:    135
 Number of total equations: -  20
 Number of slack variables: -  10
 ---------------------------------------
 Degrees of freedom       :    105
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj:  4.48E+04 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi:    2 Dpth:    1 Lvs:    2 Obj:  4.48E+04 Gap:       NaN
Iter:     3 I:  0 Tm:      0.00 NLPi:    3 Dpth:    1 Lvs:    4 Obj:  4.74E+04 Ga